# Lecture 09 - Regression

http://www.statsmodels.org/stable/index.html

conda install statsmodels

or 

pip install --upgrade --no-deps statsmodels

## A very basic example

### The import statements

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
import pandas

### Load the data

http://vincentarelbundock.github.io/Rdatasets/datasets.html

http://vincentarelbundock.github.io/Rdatasets/doc/HistData/Guerry.html

In [ ]:
df = sm.datasets.get_rdataset("Guerry", "HistData").data

df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()

df.head()

### Create a regression model

Our model here trys to determine the Lottery based on Literacy, wealth, and the region

Lottery
* Per capita wager on Royal Lottery. Ranked ratio of the proceeds bet on the royal lottery to population— Average for the years 1822-1826. Source: A1 (Compte rendus par le ministre des finances)

Literacy
* Percent Read & Write: Percent of military conscripts who can read and write. Source: A2

Wealth
* Per capita tax on personal property. A ranked index based on taxes on personal and movable property per inhabitant. Source: A1

Region
* Region of France ('N'='North', 'S'='South', 'E'='East', 'W'='West', 'C'='Central'). Corsica is coded as NA

In [ ]:
mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)

res = mod.fit()

In [ ]:
print(res.summary())

### Categorical Variables

Denoted c(*)

In [ ]:
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region)', data=df).fit()

In [ ]:
print(res.params)
print(res.summary())

### Removing variables

The “-” sign can be used to remove columns/variables. For instance, we can remove the intercept from a model by:

In [ ]:
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region) -1 ', data=df).fit()

print(res.params)
print(res.summary())

### Multiplicative interactions

”:” adds a new column to the design matrix with the product of the other two columns. “*” will also include the individual columns that were multiplied together:

In [ ]:
res1 = smf.ols(formula='Lottery ~ Literacy : Wealth - 1', data=df).fit()

res2 = smf.ols(formula='Lottery ~ Literacy * Wealth - 1', data=df).fit()

In [ ]:
print(res1.params)

print(res1.summary())

In [ ]:
print(res2.params)

print(res2.summary())

## Another way to do the regression

Using numpy arrays instead of formulas

In [ ]:
import numpy as np
import statsmodels.api as sm

# Generate artificial data (2 regressors + constant)
nobs = 100
X = np.random.random((nobs, 2))
X = sm.add_constant(X)
beta = [1, .1, .5]
e = np.random.random(nobs)
y = np.dot(X, beta) + e

In [ ]:
# Fit regression model
results = sm.OLS(y, X).fit()

# Inspect the results
print(results.summary())

## ANOVA

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
moore = sm.datasets.get_rdataset("Moore", "car", cache=True) # load
data = moore.data
data = data.rename(columns={"partner.status" :
                            "partner_status"}) # make name pythonic
moore_lm = ols('conformity ~ C(fcategory, Sum)*C(partner_status, Sum)',
                data=data).fit()
table = sm.stats.anova_lm(moore_lm, typ=2) # Type 2 ANOVA DataFrame
print(table)

## Plotting

In [ ]:
%matplotlib inline

from __future__ import print_function
from statsmodels.compat import lzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

### Load Duncan's Prestige Dataset

In [ ]:
prestige = sm.datasets.get_rdataset("Duncan", "car", cache=True).data

prestige.head()

### Run OLS regression

In [ ]:
prestige_model = ols("prestige ~ income + education", data=prestige).fit()

print(prestige_model.summary())

### Create an Influence Plot

Influence plots show the (externally) studentized residuals vs. the leverage of each observation as measured by the hat matrix.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.influence_plot(prestige_model, ax=ax, criterion="cooks")

### Partial Regression Plots

This plot is showing how much effect income has on prestige when the effects of education and income are removed.

http://www.statsmodels.org/stable/generated/statsmodels.graphics.regressionplots.plot_partregress.html

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.plot_partregress("prestige", "income", ["income", "education"], data=prestige, ax=ax)

This plot is showing how much effect income has on prestige when the effects of education are removed.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.plot_partregress("prestige", "income", ["education"], data=prestige, ax=ax)

You can see that the partial regression plot confirms the influence of conductor, minister, and RR.engineer on the partial relationship between income and prestige. The cases greatly decrease the effect of income on prestige. Dropping these cases confirms this.


In [ ]:
subset = ~prestige.index.isin(["conductor", "RR.engineer", "minister"])
prestige_model2 = ols("prestige ~ income + education", data=prestige, subset=subset).fit()
print(prestige_model2.summary())

For a quick check of all the regressors, you can use plot_partregress_grid. These plots will not label the 
points, but you can use them to identify problems and then use plot_partregress to get more information.

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(prestige_model, fig=fig)